
#### Qlunc tutorial by Francisco Costa

# **Tutorial 2**
## Designing two lidar devices and compare against each other

Along [Tutorial 1](file:///C:/Users/fcosta/SWE_LOCAL/Sphinx4Qlunc/_build/Tutorial1.html) we've learnt how to create a lidar device. Let's design two lidar devices and compare them. These devices differ from each other just in the scanner head, so we will build up two optic modules by changing the scanner component.

In [1]:
import os
os.chdir('../')
# importing  uncertainty functions
import Utils.Qlunc_Help_standAlone as SA, UQ_Functions.UQ_Photonics_Classes as uphc,UQ_Functions.UQ_Optics_Classes as uopc, UQ_Functions.UQ_Power_Classes as upwc,UQ_Functions.UQ_Lidar_Classes as ulc, UQ_Functions.UQ_ProbeVolume_Classes as upbc,UQ_Functions.UQ_Data_processing_Classes as uprm

from Utils.Qlunc_ImportModules import *

## Reading data from yaml files
Let's design two different lidar devices. To do so we will read data from two differet yaml files: `yaml_inputs_file_2_1.yml` and `yaml_inputs_file_2_2.yml`

In [2]:
# Reading data from .yml files:

# Reading first lidar data and store it into Qlunc_yaml_inputs_1 variable
with open (r'./Tutorials/yaml_inputs_file_2_1.yml','r') as file:
    Qlunc_yaml_inputs_1 = {}
    docs = yaml.load_all(file, Loader=yaml.FullLoader)
    for doc in docs:      
        for k, v in doc.items():     
            Qlunc_yaml_inputs_1.setdefault(k,v)

# Reading second lidar data and store it into Qlunc_yaml_inputs_2 variable
with open (r'./Tutorials/yaml_inputs_file_2_2.yml','r') as file:
    Qlunc_yaml_inputs_2 = {}
    docs = yaml.load_all(file, Loader=yaml.FullLoader)
    for doc in docs:      
        for k, v in doc.items():     
            Qlunc_yaml_inputs_2.setdefault(k,v)

Let's have a look to the data:

In [3]:
Qlunc_yaml_inputs_2

{'Main_directory': './Tutorials',
 'Project': 'Gandia_2',
 'Components': {'Scanner': {'Name': 'Scanner_Yaml_2',
   'Origin': [0, 0, 0],
   'Pattern time': 20,
   'Single point measuring time': 2,
   'Lissajous parameters': [100, 50, 50, 3, 3],
   'Sample rate': 0,
   'Type': 'VAD',
   'Pattern': 'None',
   'stdv focus distance': 1,
   'stdv Cone angle': 2,
   'stdv Azimuth': 3,
   'Focus distance': [30],
   'Cone angle': [15],
   'Azimuth': [0, 360],
   'x': [160, 160, 160, 160],
   'y': [50, -50, -50, 50],
   'z': [46.21070722778538, 46.21070722778538, -46.21070722778537, -85],
   'Uncertainty function': 'uopc.UQ_Scanner'},
  'Optical Circulator': {'Name': 'OC_Yaml_2',
   'Insertion loss': 0.8,
   'SNR': 60,
   'Uncertainty function': 'uopc.UQ_OpticalCirculator'},
  'Telescope': {'Name': 'Telescope_YAML_2',
   'Aperture': 0.02,
   'Focal length': 0.5,
   'Fiber-lens distance': 0.0024,
   'Fiber-lens offset': 0.498,
   'Effective radius telescope': 0.0166,
   'Output beam radius': 0.02

In [4]:
Qlunc_yaml_inputs_1

{'Main_directory': './Tutorials',
 'Project': 'Gandia',
 'Components': {'Scanner': {'Name': 'Scanner_Yaml_1',
   'Origin': [0, 0, 0],
   'Pattern time': 20,
   'Single point measuring time': 2,
   'Lissajous parameters': [100, 50, 50, 3, 3],
   'Sample rate': 0,
   'Type': 'VAD',
   'Pattern': 'None',
   'stdv focus distance': 1,
   'stdv Cone angle': 2,
   'stdv Azimuth': 3,
   'Focus distance': [75],
   'Cone angle': [15],
   'Azimuth': [0, 360],
   'x': [160, 160, 160, 160],
   'y': [50, -50, -50, 50],
   'z': [46.21070722778538, 46.21070722778538, -46.21070722778537, -85],
   'Uncertainty function': 'uopc.UQ_Scanner'},
  'Optical Circulator': {'Name': 'OC_Yaml_1',
   'Insertion loss': 0.8,
   'SNR': 60,
   'Uncertainty function': 'uopc.UQ_OpticalCirculator'},
  'Telescope': {'Name': 'Telescope_YAML_1',
   'Aperture': 0.02,
   'Focal length': 0.5,
   'Fiber-lens distance': 0.0024,
   'Fiber-lens offset': 0.498,
   'Effective radius telescope': 0.0166,
   'Output beam radius': 0.02,


## Creating python classes:
We create the python classes by running the Qlunc Core: `Qlunc_Classes.py`. Since both `.yml` files are in the same directory just have to chose where do we want to take the `WorkingExample_Qlunc_Classes.py` path from.

In [5]:
# Execute Qlunc_Classes.py (creating classes for lidar 'objects'):
exec(open(Qlunc_yaml_inputs_2['Main_directory']+'/WorkingExample_Tutorial.py').read())  

The main directory is taken from the yaml file, as well as the rest of the lidar parameters, including plotting flags which select what we want to plot:


## Creating scanner heads:
Then is the moment to create a scanner object for each lidar using the different input data parameters

In [6]:
# Instantiating scanner class to create a Scanner virtual-twin:

Scanner_1           = scanner(name            = Qlunc_yaml_inputs_1['Components']['Scanner']['Name'],           # Introduce your scanner name.
                            scanner_type    = Qlunc_yaml_inputs_1['Components']['Scanner']['Type'],
                            origin          = Qlunc_yaml_inputs_1['Components']['Scanner']['Origin'],         # Origin (coordinates of the lidar deployment).
                            pattern         = Qlunc_yaml_inputs_1['Components']['Scanner']['Pattern'],
                            lissajous_param = Qlunc_yaml_inputs_1['Components']['Scanner']['Lissajous parameters'],
                            sample_rate     = Qlunc_yaml_inputs_1['Components']['Scanner']['Sample rate'],    # for now introduce it in [degrees].
                            time_pattern    = Qlunc_yaml_inputs_1['Components']['Scanner']['Pattern time'],
                            time_point      = Qlunc_yaml_inputs_1['Components']['Scanner']['Single point measuring time'],      
                           # This values for focus distance, cone_angle and azimuth define a typical VAD scanning sequence:
                               # I changed azimuth calculations because with "np.arange" we do not capture the last point in the pattern. "np.arange does not include the last point"; np.linspace capture all the points.
                               # Furthermore, once the time of the pattern is included in the pattern, we will do calculations based on the n° of points yielded by the ratio: time_pattern[sec]/time_point[sec/point]
                               # HAve to decide if wnat np.arange or np.linspace here (azimuth). If 360° is chosen for azimuth, np.arange works, but np.linspace doesn't
                            azimuth         = np.array(np.arange(Qlunc_yaml_inputs_1['Components']['Scanner']['Azimuth'][0],                                                  
                                                                  Qlunc_yaml_inputs_1['Components']['Scanner']['Azimuth'][1],
                                                                  math.floor((Qlunc_yaml_inputs_1['Components']['Scanner']['Azimuth'][1]-Qlunc_yaml_inputs_1['Components']['Scanner']['Azimuth'][0])/(Qlunc_yaml_inputs_1['Components']['Scanner']['Pattern time']/Qlunc_yaml_inputs_1['Components']['Scanner']['Single point measuring time'])))), # Azimuth angle in [degrees].
                            
                            # azimuth         = np.array(np.linspace(Qlunc_yaml_inputs['Components']['Scanner']['Azimuth'][0],                                                  
                            #                                         Qlunc_yaml_inputs['Components']['Scanner']['Azimuth'][1],
                            #                                         math.floor(Qlunc_yaml_inputs['Components']['Scanner']['Pattern time']/Qlunc_yaml_inputs['Components']['Scanner']['Single point measuring time']))), # Azimuth angle in [degrees].                                  
                            focus_dist      = np.tile(Qlunc_yaml_inputs_1['Components']['Scanner']['Focus distance'],(1,len(np.linspace(Qlunc_yaml_inputs_1['Components']['Scanner']['Azimuth'][0],                                                  
                                                                   Qlunc_yaml_inputs_1['Components']['Scanner']['Azimuth'][1],
                                                                   math.floor(Qlunc_yaml_inputs_1['Components']['Scanner']['Pattern time']/Qlunc_yaml_inputs_1['Components']['Scanner']['Single point measuring time'])))))[0],   # Focus distance in [meters]                                                                                                
                            cone_angle      = np.tile(Qlunc_yaml_inputs_1['Components']['Scanner']['Cone angle'],(1,len(np.linspace(Qlunc_yaml_inputs_1['Components']['Scanner']['Azimuth'][0],                                                  
                                                                   Qlunc_yaml_inputs_1['Components']['Scanner']['Azimuth'][1],
                                                                   math.floor(Qlunc_yaml_inputs_1['Components']['Scanner']['Pattern time']/Qlunc_yaml_inputs_1['Components']['Scanner']['Single point measuring time'])))))[0],   # Cone angle in [degrees].
                            x               = np.array(Qlunc_yaml_inputs_1['Components']['Scanner']['x']),
                            y               = np.array(Qlunc_yaml_inputs_1['Components']['Scanner']['y']),
                            z               = np.array(Qlunc_yaml_inputs_1['Components']['Scanner']['z']),
                            stdv_focus_dist = Qlunc_yaml_inputs_1['Components']['Scanner']['stdv focus distance'],                 # Focus distance standard deviation in [meters].
                            stdv_cone_angle = Qlunc_yaml_inputs_1['Components']['Scanner']['stdv Cone angle'],                 # Cone angle standard deviation in [degrees].
                            stdv_azimuth    = Qlunc_yaml_inputs_1['Components']['Scanner']['stdv Azimuth'],                 # Azimuth angle standard deviation in [degrees].
                            unc_func        = uopc.UQ_Scanner) #eval(Qlunc_yaml_inputs['Components']['Scanner']['Uncertainty function']) )    # here you put the function describing your scanner uncertainty. 

Scanner_2           = scanner(name            = Qlunc_yaml_inputs_2['Components']['Scanner']['Name'],           # Introduce your scanner name.
                            scanner_type    = Qlunc_yaml_inputs_2['Components']['Scanner']['Type'],
                            origin          = Qlunc_yaml_inputs_2['Components']['Scanner']['Origin'],         # Origin (coordinates of the lidar deployment).
                            pattern         = Qlunc_yaml_inputs_2['Components']['Scanner']['Pattern'],
                            lissajous_param = Qlunc_yaml_inputs_2['Components']['Scanner']['Lissajous parameters'],
                            sample_rate     = Qlunc_yaml_inputs_2['Components']['Scanner']['Sample rate'],    # for now introduce it in [degrees].
                            time_pattern    = Qlunc_yaml_inputs_2['Components']['Scanner']['Pattern time'],
                            time_point      = Qlunc_yaml_inputs_2['Components']['Scanner']['Single point measuring time'],      
                           # This values for focus distance, cone_angle and azimuth define a typical VAD scanning sequence:
                               # I changed azimuth calculations because with "np.arange" we do not capture the last point in the pattern. "np.arange does not include the last point"; np.linspace capture all the points.
                               # Furthermore, once the time of the pattern is included in the pattern, we will do calculations based on the n° of points yielded by the ratio: time_pattern[sec]/time_point[sec/point]
                               # HAve to decide if wnat np.arange or np.linspace here (azimuth). If 360° is chosen for azimuth, np.arange works, but np.linspace doesn't
                            azimuth         = np.array(np.arange(Qlunc_yaml_inputs_2['Components']['Scanner']['Azimuth'][0],                                                  
                                                                  Qlunc_yaml_inputs_2['Components']['Scanner']['Azimuth'][1],
                                                                  math.floor((Qlunc_yaml_inputs_2['Components']['Scanner']['Azimuth'][1]-Qlunc_yaml_inputs_2['Components']['Scanner']['Azimuth'][0])/(Qlunc_yaml_inputs_2['Components']['Scanner']['Pattern time']/Qlunc_yaml_inputs_2['Components']['Scanner']['Single point measuring time'])))), # Azimuth angle in [degrees].
                            
                            # azimuth         = np.array(np.linspace(Qlunc_yaml_inputs['Components']['Scanner']['Azimuth'][0],                                                  
                            #                                         Qlunc_yaml_inputs['Components']['Scanner']['Azimuth'][1],
                            #                                         math.floor(Qlunc_yaml_inputs['Components']['Scanner']['Pattern time']/Qlunc_yaml_inputs['Components']['Scanner']['Single point measuring time']))), # Azimuth angle in [degrees].                                  
                            focus_dist      = np.tile(Qlunc_yaml_inputs_2['Components']['Scanner']['Focus distance'],(1,len(np.linspace(Qlunc_yaml_inputs_2['Components']['Scanner']['Azimuth'][0],                                                  
                                                                   Qlunc_yaml_inputs_2['Components']['Scanner']['Azimuth'][1],
                                                                   math.floor(Qlunc_yaml_inputs_2['Components']['Scanner']['Pattern time']/Qlunc_yaml_inputs_2['Components']['Scanner']['Single point measuring time'])))))[0],   # Focus distance in [meters]                                                                                                
                            cone_angle      = np.tile(Qlunc_yaml_inputs_2['Components']['Scanner']['Cone angle'],(1,len(np.linspace(Qlunc_yaml_inputs_2['Components']['Scanner']['Azimuth'][0],                                                  
                                                                   Qlunc_yaml_inputs_2['Components']['Scanner']['Azimuth'][1],
                                                                   math.floor(Qlunc_yaml_inputs_2['Components']['Scanner']['Pattern time']/Qlunc_yaml_inputs_2['Components']['Scanner']['Single point measuring time'])))))[0],   # Cone angle in [degrees].
                            x               = np.array(Qlunc_yaml_inputs_2['Components']['Scanner']['x']),
                            y               = np.array(Qlunc_yaml_inputs_2['Components']['Scanner']['y']),
                            z               = np.array(Qlunc_yaml_inputs_2['Components']['Scanner']['z']),
                            stdv_focus_dist = Qlunc_yaml_inputs_2['Components']['Scanner']['stdv focus distance'],                 # Focus distance standard deviation in [meters].
                            stdv_cone_angle = Qlunc_yaml_inputs_2['Components']['Scanner']['stdv Cone angle'],                 # Cone angle standard deviation in [degrees].
                            stdv_azimuth    = Qlunc_yaml_inputs_2['Components']['Scanner']['stdv Azimuth'],                 # Azimuth angle standard deviation in [degrees].
                            unc_func        = uopc.UQ_Scanner) #eval(Qlunc_yaml_inputs['Components']['Scanner']['Uncertainty function']) )    # here you put the function describing your scanner uncertainty. 


Created new scanner: Scanner_Yaml_1
Created new scanner: Scanner_Yaml_2


## Optical circulator:
Once we have created the different scanning heads we can create the optical circulator. Since they use the same optical circulator we just have to create one for both lidars. 

In [7]:
#Optical Circulator:

Optical_circulator = optical_circulator (name           = Qlunc_yaml_inputs_1['Components']['Optical Circulator']['Name'],       # Introduce your Optical circulator name.
                                         insertion_loss = Qlunc_yaml_inputs_1['Components']['Optical Circulator']['Insertion loss'],                        # In [dB]. Insertion loss parameters.
                                         SNR            = Qlunc_yaml_inputs_1['Components']['Optical Circulator']['SNR'], # [dB] SNR optical circulator
                                         unc_func       = uopc.UQ_OpticalCirculator) #eval(Qlunc_yaml_inputs['Components']['Optical Circulator']['Uncertainty function']))  # Function describing your scanner uncertainty.  Further informaion in "UQ_Optics_Classes.py" comments.


Created new optical circulator: OC_Yaml_1


## Optics modules:
Create the optics modules by introducing scanners and optical circulator in them (by instantiating the optics `class`):

In [8]:
# Optics Module lidar1:

Optics_Module_1 =  optics (name               = Qlunc_yaml_inputs_1['Modules']['Optics Module']['Name'],     # Introduce your Optics Module name.
                         scanner            = Scanner_1,# Scanner instance (in this example "Scanner") or "None". "None" means that you don´t want to include Scanner in Optics Module, either in uncertainty calculations.
                         optical_circulator = Optical_circulator,# Optical Circulator instance (in this example "Optical_circulator") or "None". "None" means that you don´t want to include Optical circulator in Optics Module, either in uncertainty calculations.
                         telescope          = None, #Telescope
                         unc_func           = uopc.sum_unc_optics) #eval(Qlunc_yaml_inputs['Modules']['Optics Module']['Uncertainty function']))
# Optics Module lidar2:
Optics_Module_2 =  optics (name               = Qlunc_yaml_inputs_2['Modules']['Optics Module']['Name'],     # Introduce your Optics Module name.
                         scanner            = Scanner_2, # Scanner instance (in this example "Scanner") or "None". "None" means that you don´t want to include Scanner in Optics Module, either in uncertainty calculations.
                         optical_circulator = Optical_circulator, # Optical Circulator instance (in this example "Optical_circulator") or "None". "None" means that you don´t want to include Optical circulator in Optics Module, either in uncertainty calculations.
                         telescope          = None,  #Telescope
                         unc_func           = uopc.sum_unc_optics)


Created new optic module: Optics_module_Yaml_1
Created new optic module: Optics_module_Yaml_2


## Photonics:

In [9]:
# Instantiating optical amplifier:

Optical_Amplifier = optical_amplifier(name             = Qlunc_yaml_inputs_1['Components']['Optical Amplifier']['Name'],        # Introduce your scanner name.
                                      NoiseFig         = Qlunc_yaml_inputs_1['Components']['Optical Amplifier']['Optical amplifier noise figure'],          # In [dB]. Can introduce it as a table from manufactures (in this example the data is taken from Thorlabs.com, in section EDFA\Graps) or introduce a single well-known value
                                      OA_Gain          = Qlunc_yaml_inputs_1['Components']['Optical Amplifier']['Optical amplifier gain'],                         # In [dB]. (in this example the data is taken from Thorlabs.com, in section EDFA\Specs)
                                      OA_BW            = Qlunc_yaml_inputs_1['Components']['Optical Amplifier']['Optical amplifier BW'],
                                      Power_interval   = np.array(np.arange(Qlunc_yaml_inputs_1['Components']['Optical Amplifier']['Power interval'][0],
                                                                            Qlunc_yaml_inputs_1['Components']['Optical Amplifier']['Power interval'][1],
                                                                            Qlunc_yaml_inputs_1['Components']['Optical Amplifier']['Power interval'][2])),
                                      unc_func         = uphc.UQ_Optical_amplifier) # Function describing Optical Amplifier uncertainty. Further informaion in "UQ_Photonics_Classes.py" comments.


Created new optical amplifier: OA_Yaml_1


Photodetector digital twin:

In [10]:
# Instantiating Photodetector:

Photodetector    = photodetector(name             = Qlunc_yaml_inputs_1['Components']['Photodetector']['Name'],                           # Introduce your photodetector name.
                                 Photo_BandWidth  = Qlunc_yaml_inputs_1['Components']['Photodetector']['Photodetector BandWidth'],        # Photodetector bandwidth
                                 Load_Resistor    = Qlunc_yaml_inputs_1['Components']['Photodetector']['Load resistor'],                  # [ohms]
                                 Photo_efficiency = Qlunc_yaml_inputs_1['Components']['Photodetector']['Photodetector efficiency'],       # Photodetector efficiency [-]
                                 Dark_Current     = Qlunc_yaml_inputs_1['Components']['Photodetector']['Dark current'],                   # [A]. Dark current in the photodetector.
                                 Photo_SignalP    = Qlunc_yaml_inputs_1['Components']['Photodetector']['Photodetector signalP'],
                                 Power_interval   = np.array(np.arange(Qlunc_yaml_inputs_1['Components']['Photodetector']['Power interval'][0],
                                                                       Qlunc_yaml_inputs_1['Components']['Photodetector']['Power interval'][1],
                                                                       Qlunc_yaml_inputs_1['Components']['Photodetector']['Power interval'][2])),# [w]. Power interval for the photodetector domain in photodetector SNR plot. 
                                 Active_Surf      = Qlunc_yaml_inputs_1['Components']['Photodetector']['Active Surface'],
                                 Gain_TIA         = Qlunc_yaml_inputs_1['Components']['Photodetector']['Gain TIA'],                    # [dB]. If there is a transimpedance amplifier.
                                 V_Noise_TIA      = Qlunc_yaml_inputs_1['Components']['Photodetector']['V Noise TIA'],                 # [V]. If there is a transimpedance amplifier.
                                 
                                 unc_func         = uphc.UQ_Photodetector) #Function describing Photodetector uncertainty. Further informaion in "UQ_Photonics_Classes.py" comments.


Created new photodetector: Photodetector_YAML_1


Photonics module digital twin:
And finally the Photonics module:


In [11]:
# Instantiating Photonics module:

Photonics_Module = photonics(name                    = Qlunc_yaml_inputs_1['Modules']['Photonics Module']['Name'],        # Introduce your Photonics module name
                             photodetector           = Photodetector,# Photodetector instance (in this example "Photodetector") or "None". "None" means that you don´t want to include photodetector in Photonics Module, either in uncertainty calculations.
                             optical_amplifier       = Optical_Amplifier,# Optical amplifier instance (in this example "OpticalAmplifier") or "None". "None" means that you don´t want to include Optical Amplifier in Photonics Module, either in uncertainty calculations.
                             laser                   = 'None', 
                             acousto_optic_modulator = 'None', 
                             unc_func                = uphc.sum_unc_photonics)


Created new photonic module: Photonics_module_Yaml_1


Lidar general inputs:

In [12]:
# Instantiating lidar general inputs

Lidar_inputs     = lidar_gral_inp(name        = Qlunc_yaml_inputs_1['Components']['Lidar general inputs']['Name'],      # Introduce the name of your lidar data folder.
                                  wave        = Qlunc_yaml_inputs_1['Components']['Lidar general inputs']['Wavelength'],                    # In [m]. Lidar wavelength.
                                  ltype       = Qlunc_yaml_inputs_1['Components']['Lidar general inputs']['Type'],
                                  yaw_error   = Qlunc_yaml_inputs_1['Components']['Lidar general inputs']['Yaw error'],                          # In [°]. Degrees of rotation around z axis because of inclinometer errors
                                  pitch_error = Qlunc_yaml_inputs_1['Components']['Lidar general inputs']['Pitch error'],                          # In [°]. Degrees of rotation around y axis
                                  roll_error  = Qlunc_yaml_inputs_1['Components']['Lidar general inputs']['Roll error'],                        # In [°]. Degrees of rotation around z axis.
                                  dataframe   = { })  # Final dataframe


Lidar_inputs_2   = lidar_gral_inp(name        = Qlunc_yaml_inputs_2['Components']['Lidar general inputs']['Name'],      # Introduce the name of your lidar data folder.
                                  wave        = Qlunc_yaml_inputs_2['Components']['Lidar general inputs']['Wavelength'],                    # In [m]. Lidar wavelength.
                                  ltype       = Qlunc_yaml_inputs_2['Components']['Lidar general inputs']['Type'],
                                  yaw_error   = Qlunc_yaml_inputs_2['Components']['Lidar general inputs']['Yaw error'],                          # In [°]. Degrees of rotation around z axis because of inclinometer errors
                                  pitch_error = Qlunc_yaml_inputs_2['Components']['Lidar general inputs']['Pitch error'],                          # In [°]. Degrees of rotation around y axis
                                  roll_error  = Qlunc_yaml_inputs_2['Components']['Lidar general inputs']['Roll error'],                        # In [°]. Degrees of rotation around z axis.
                                  dataframe   = { })  # Final dataframe



Created new lidar general inputs: Lidar_gral_inputs_Yaml_1
Created new lidar general inputs: Lidar_gral_inputs_Yaml_2


Atmospheric scenarios:

We can create different atmospheric scenarios under which the lidar uncertainties are assessed. Thus, single or multiple scenarios can be evaluated either including single or time-seried atmospheric variables values.

In [13]:
# Instantiating atmospheric scenario(s): Assuming both lidars ahve the same atmospheric 
Atmospheric_TimeSeries = Qlunc_yaml_inputs_1['Atmospheric_inputs']['TimeSeries']  
                                                                           
if Atmospheric_TimeSeries:
    Atmos_TS_FILE           = '../metadata/AtmosphericData/'+Qlunc_yaml_inputs_1['Atmospheric_inputs']['Atmos_TS_FILE']
    AtmosphericScenarios_TS = pd.read_csv(Atmos_TS_FILE,delimiter=';',decimal=',')
    Atmospheric_inputs = {
                          'temperature' : list(AtmosphericScenarios_TS.loc[:,'T']),    
                          'humidity'    : list(AtmosphericScenarios_TS.loc[:,'H']),    
                          'rain'        : list(AtmosphericScenarios_TS.loc[:,'rain']),
                          'fog'         : list(AtmosphericScenarios_TS.loc[:,'fog']),
                          'time'        : list(AtmosphericScenarios_TS.loc[:,'t'])     
                          } 
    Atmospheric_Scenario = atmosphere(name        = 'Atmosphere1',
                                      temperature = Atmospheric_inputs['temperature'])
else:    

    Atmospheric_Scenario = atmosphere(name        = 'Atmosphere1',
                                      temperature = Qlunc_yaml_inputs_1['Atmospheric_inputs']['Temperature'])

Created new atmosphere: Atmosphere1


In [14]:
#%% Data processing methods

# Wind field reconstruction model
WFR_M = wfr (name                 = Qlunc_yaml_inputs_1['WFR model']['Name'],
             reconstruction_model = Qlunc_yaml_inputs_1['WFR model']['Model'],
             unc_func             = uprm.UQ_WFR)

# Data filtering method
Filt_M = filtering_method (name        = Qlunc_yaml_inputs_1['Filtering method']['Name'],
                           filt_method = Qlunc_yaml_inputs_1['Filtering method']['Method'],
                           unc_func    = 'uprm.UQ_WFR')
Probe_Volume = probe_volume (name                       = Qlunc_yaml_inputs_1['Probe Volume']['Name'],
                             extinction_coef            = Qlunc_yaml_inputs_1['Probe Volume']['Extinction coeficient'],
                             unc_func                   = upbc.UQ_Probe_volume)


Selected wfr model: None terrain
Selected filtering model: CNR 
Class "Probe volume" created


Now we build up the lidar devices by putting all components together:

In [15]:
Lidar_1 = lidar(name         = Qlunc_yaml_inputs_1['Lidar']['Name'], # Introduce the name of your lidar device.
                photonics    = Photonics_Module,# Introduce the name of your photonics module.
                optics       = Optics_Module_1,# Introduce the name of your optics module.
                power        = None, # Introduce the name of your power module. NOT IMPLEMENTED YET!
                wfr_model    = None,
                filt_method  = None,
                probe_volume = Probe_Volume, 
                lidar_inputs = Lidar_inputs,# Introduce lidar general inputs
                unc_func     = ulc.sum_unc_lidar)#eval(Qlunc_yaml_inputs_1['Lidar']['Uncertainty function'])) # Function estimating lidar global uncertainty

Lidar_2 = lidar(name         = Qlunc_yaml_inputs_2['Lidar']['Name'],                       # Introduce the name of your lidar device.
                photonics    = Photonics_Module,# Introduce the name of your photonics module.
                optics       = Optics_Module_2,# Introduce the name of your optics module.
                power        = None,         # Introduce the name of your power module. NOT IMPLEMENTED YET!
                wfr_model    = None,
                filt_method  = None,
                probe_volume = Probe_Volume, 
                lidar_inputs = Lidar_inputs_2,# Introduce lidar general inputs
                unc_func     = ulc.sum_unc_lidar )#eval(Qlunc_yaml_inputs_2['Lidar']['Uncertainty function'])) # Function estimating lidar global uncertainty


Created new lidar device: Caixa1_1
Created new lidar device: Caixa1_2


### Asking for the scanner uncertainties and plot the results

In [16]:
os.chdir('C:\SWE_LOCAL\GIT_Qlunc')

In [ ]:
Scanner_Data1,ScDf1 = Lidar_1.optics.scanner.Uncertainty(Lidar_1,Atmospheric_Scenario,cts,Qlunc_yaml_inputs_1)
Scanner_Data2,ScDf2 = Lidar_2.optics.scanner.Uncertainty(Lidar_2,Atmospheric_Scenario,cts,Qlunc_yaml_inputs_2) 

In [ ]:
# Plotting parameters:
plot_param={
            'axes_label_fontsize' : 16,
            'textbox_fontsize'    : 14,
            'title_fontsize'      : 18,
            'suptitle_fontsize'   : 23,
            'legend_fontsize'     : 12,
            'xlim'                : [-25,25],
            'ylim'                : [-25,25],
            'zlim'                : [0,90],
            'markersize'          : 8,
            'markersize_lidar'    : 12,
            'marker'              : '.',
            'markerTheo'          : '.b',
            'tick_labelrotation'  : 45,
            'Qlunc_version'       : 'Qlunc Version - 0.9',
            'Props'               : dict(boxstyle='round', facecolor='wheat', alpha=0.5)
           }

In [ ]:
# Creating the figure and the axes
fig,axs = plt.subplots()  
axs=plt.axes(projection='3d')

# Plotting

# First lidar (Lidar_1)
line0,=axs.plot([Lidar_1.optics.scanner.origin[0]],[Lidar_1.optics.scanner.origin[1]],[Lidar_1.optics.scanner.origin[2]],'ob',label='{} coordinates [{},{},{}]'.format(Lidar_1.LidarID,Lidar_1.optics.scanner.origin[0],Lidar_1.optics.scanner.origin[1],Lidar_1.optics.scanner.origin[2]),markersize=plot_param['markersize_lidar'])
line1,=axs.plot(Scanner_Data1['MeasPoint_Coordinates'][0],Scanner_Data1['MeasPoint_Coordinates'][1],Scanner_Data1['MeasPoint_Coordinates'][2],plot_param['markerTheo'],markersize=plot_param['markersize'],label='Theoretical measuring point')
line2,=axs.plot(Scanner_Data1['NoisyMeasPoint_Coordinates'][0],Scanner_Data1['NoisyMeasPoint_Coordinates'][1],Scanner_Data1['NoisyMeasPoint_Coordinates'][2],plot_param['marker'],markersize=plot_param['markersize'],label='Distance error [m] = {0:.3g}$\pm${1:.3g}'.format(np.mean(Scanner_Data1['Simu_Mean_Distance_Error']),np.mean(Scanner_Data1['STDV_Distance'])))

# Second lidar(Lidar_2)
axs.plot([Lidar_2.optics.scanner.origin[0]],[Lidar_2.optics.scanner.origin[1]],[Lidar_2.optics.scanner.origin[2]],'ob',label='{} coordinates [{},{},{}]'.format(Lidar_2.LidarID,Lidar_2.optics.scanner.origin[0],Lidar_2.optics.scanner.origin[1],Lidar_2.optics.scanner.origin[2]),markersize=plot_param['markersize_lidar'])
axs.plot(Scanner_Data2['MeasPoint_Coordinates'][0],Scanner_Data2['MeasPoint_Coordinates'][1],Scanner_Data2['MeasPoint_Coordinates'][2],plot_param['markerTheo'],markersize=plot_param['markersize'],label='Theoretical measuring point')
line3,=axs.plot(Scanner_Data2['NoisyMeasPoint_Coordinates'][0],Scanner_Data2['NoisyMeasPoint_Coordinates'][1],Scanner_Data2['NoisyMeasPoint_Coordinates'][2],plot_param['marker'],markersize=plot_param['markersize'],label='Distance error [m] = {0:.3g}$\pm${1:.3g}'.format(np.mean(Scanner_Data2['Simu_Mean_Distance_Error']),np.mean(Scanner_Data2['STDV_Distance'])))

# Setting labels, legend, title and axes limits:
axs.set_xlabel('x [m]',fontsize=plot_param['axes_label_fontsize'])
axs.set_ylabel('y [m]',fontsize=plot_param['axes_label_fontsize'])
axs.set_zlabel('z [m]',fontsize=plot_param['axes_label_fontsize'])
axs.set_title('Scanner Pointing accuracy',fontsize=plot_param['title_fontsize'])
axs.set_xlim3d(plot_param['xlim'][0],plot_param['xlim'][1])
axs.set_ylim3d(plot_param['ylim'][0],plot_param['ylim'][1])
axs.set_zlim3d(plot_param['zlim'][0],plot_param['zlim'][1])

# legends
legend = plt.legend(handles=[line0,line1,line2,line3], loc='upper right')
plt.gca().add_artist(legend)
plt.rcParams['figure.figsize'] = [25, 13]
